
<style>
    contato{fontsize=10}
</style>

<h3><strong>Trabalho Prático 1 - Computação Gráfica</strong></h3>
<h4><strong>Autores:</strong>
    <ul>
        <li>Gustavo Rafael Nunes Durães</li>
                <contato>Email: grnd@aluno.ifnmg.edu.br     Telefone: (38) 99737-7689
                <p></p>
        <li>Lucas Flávio Gabrich Marinho</li>
                Email: lfgm@aluno.ifnmg.edu.br     Telefone: (38) 99941-8775
    </ul>
</h4>


<ol><strong>Instruções de Execução:</strong>
    <li>Garanta que as devidas bibliotecas estão intaladas.</li>
    <li>O arquivo "entrada.xml" deve estar na mesma pasta</li>
    <li>Execute todas as celulas (Run all) ou execute-as em sequência.</li>
    <li>A última celula iniciará o script que funciona de forma automática.</li>
    <li>A janela mostrando os objetos será aberta (verifique a barra de tarefas).</li>
    <li>O Arquivo "saida.xml" com os resultados das transformações será salvo na mesma pasta do script.</li>
    <li>Para encerrar a execução, basta fechar a janela que foi aberta.</li>
</ol>
<p><strong>OBS:</strong> Caso deseje executar novamente é possivel que o Kernel Python pare, não uma encontramos uma solução para isso, mas basta reiniciar o Kernel Python que o script deve executar sem problemas.</p>

In [1]:
import xml.etree.ElementTree as ET
import PyQt5 # type: ignore
from PyQt5.QtWidgets import QApplication, QMainWindow, QGraphicsScene, QGraphicsView, QGraphicsLineItem, QGraphicsPolygonItem, QGraphicsEllipseItem # type: ignore
from PyQt5.QtCore import Qt, QPointF # type: ignore
from PyQt5.QtGui import QPolygonF # type: ignore
from PyQt5 import QtGui # type: ignore


In [2]:
class Window:
    def __init__(self, xwmin, ywmin, xwmax, ywmax):
        self.xwmin = xwmin
        self.ywmin = ywmin
        self.xwmax = xwmax
        self.ywmax = ywmax

In [3]:
class Ponto2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [4]:
class Reta2D:
    def __init__(self, ponto1, ponto2):
        self.ponto1 = ponto1
        self.ponto2 = ponto2

In [5]:
class Poligono2D:
    def __init__(self, pontos):
        self.pontos = pontos

In [6]:
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Leitura do arquivo XML
        tree = ET.parse('entrada.xml')
        root = tree.getroot()

        # Leitura dos dados da viewport
        viewport_data = root.find('viewport')
        vpmin = viewport_data.find('vpmin')
        vpmax = viewport_data.find('vpmax')

        vpmin_x = float(vpmin.attrib['x'])
        vpmin_y = float(vpmin.attrib['y'])
        vpmax_x = float(vpmax.attrib['x'])
        vpmax_y = float(vpmax.attrib['y'])

        # Leitura dos dados da window
        window_data = root.find('window')
        wmin = window_data.find('wmin')
        wmax = window_data.find('wmax')

        wmin_x = float(wmin.attrib['x'])
        wmin_y = float(wmin.attrib['y'])
        wmax_x = float(wmax.attrib['x'])
        wmax_y = float(wmax.attrib['y'])

        self.window = Window(wmin_x, wmin_y, wmax_x, wmax_y)

        # Lista para armazenar os objetos transformados
        self.objetos_transformados = []

        # Transformação dos objetos para o sistema de coordenadas da viewport
        w_width = self.window.xwmax - self.window.xwmin
        w_height = self.window.ywmax - self.window.ywmin
        vp_width = vpmax_x - vpmin_x
        vp_height = vpmax_y - vpmin_y

        # Considerar a margem extra na viewport
        vpmin_x -= 10
        vpmin_y -= 10
        vpmax_x -= 10
        vpmax_y -= 10

        for child in root:
            if child.tag == 'ponto':
                x = float(child.attrib['x'])
                y = float(child.attrib['y'])
                x_vp = ((x - self.window.xwmin) / w_width) * vp_width - vpmin_x
                y_vp = ((self.window.ywmax - y) / w_height) * vp_height - vpmin_y
                self.objetos_transformados.append(Ponto2D(x_vp, y_vp))
            elif child.tag == 'reta':
                ponto1 = child.find('ponto[1]')
                ponto2 = child.find('ponto[2]')
                x1 = float(ponto1.attrib['x'])
                y1 = float(ponto1.attrib['y'])
                x2 = float(ponto2.attrib['x'])
                y2 = float(ponto2.attrib['y'])
                x1_vp = ((x1 - self.window.xwmin) / w_width) * vp_width - vpmin_x
                y1_vp = ((self.window.ywmax - y1) / w_height) * vp_height - vpmin_y
                x2_vp = ((x2 - self.window.xwmin) / w_width) * vp_width - vpmin_x
                y2_vp = ((self.window.ywmax - y2) / w_height) * vp_height - vpmin_y
                self.objetos_transformados.append(Reta2D(Ponto2D(x1_vp, y1_vp), Ponto2D(x2_vp, y2_vp)))
            elif child.tag == 'poligono':
                pontos = []
                for ponto in child.findall('ponto'):
                    x = float(ponto.attrib['x'])
                    y = float(ponto.attrib['y'])
                    x_vp = ((x - self.window.xwmin) / w_width) * vp_width - vpmin_x
                    y_vp = ((self.window.ywmax - y) / w_height) * vp_height - vpmin_y
                    pontos.append(Ponto2D(x_vp, y_vp))
                self.objetos_transformados.append(Poligono2D(pontos))

            
        # Geração do arquivo de saída em formato XML
        filename = 'saida.xml'
        root_output = ET.Element('saida')

        for obj in self.objetos_transformados:
            if isinstance(obj, Ponto2D):
                ponto = ET.SubElement(root_output, 'ponto')
                ponto.attrib['x'] = str(obj.x)
                ponto.attrib['y'] = str(obj.y)
            elif isinstance(obj, Reta2D):
                reta = ET.SubElement(root_output, 'reta')
                ponto1 = ET.SubElement(reta, 'ponto')
                ponto1.attrib['x'] = str(obj.ponto1.x)
                ponto1.attrib['y'] = str(obj.ponto1.y)
                ponto2 = ET.SubElement(reta, 'ponto')
                ponto2.attrib['x'] = str(obj.ponto2.x)
                ponto2.attrib['y'] = str(obj.ponto2.y)
            elif isinstance(obj, Poligono2D):
                poligono = ET.SubElement(root_output, 'poligono')
                for ponto in obj.pontos:
                    ponto_xml = ET.SubElement(poligono, 'ponto')
                    ponto_xml.attrib['x'] = str(ponto.x)
                    ponto_xml.attrib['y'] = str(ponto.y)

        tree_output = ET.ElementTree(root_output)
        tree_output.write(filename)

        # Criação da interface gráfica
        self.scene = QGraphicsScene()
        self.view = QGraphicsView(self.scene)
        self.setCentralWidget(self.view)

        # Exibição dos objetos na viewport
        for obj in self.objetos_transformados:
            if isinstance(obj, Ponto2D):
                item = QGraphicsEllipseItem(obj.x - 2, obj.y - 2, 4, 4)
                item.setBrush(Qt.black)
                self.scene.addItem(item)
            elif isinstance(obj, Reta2D):
                item = QGraphicsLineItem(obj.ponto1.x, obj.ponto1.y, obj.ponto2.x, obj.ponto2.y)
                self.scene.addItem(item)
            elif isinstance(obj, Poligono2D):
                pontos = [QPointF(ponto.x, ponto.y) for ponto in obj.pontos]
                poligono = QGraphicsPolygonItem()
                poligono.setPolygon(QPolygonF(pontos))

                self.scene.addItem(poligono)

In [7]:
if __name__ == '__main__':
    app = QApplication([])
    window = MyWindow()
    window.show()
    app.exec_()